5.0 explore base tree

**5.1 optimize xgb (it needs scale)** 

### 0/ Import

In [16]:
# Basic
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# preprocess, split and metric
from sklearn.metrics import confusion_matrix,classification_report,roc_auc_score,plot_confusion_matrix,precision_recall_curve,roc_curve,make_scorer, f1_score, precision_score, recall_score, accuracy_score
#from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

# classifier algo
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier,RandomForestClassifier,GradientBoostingClassifier,VotingClassifier,AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.svm import SVC

#random
np.random.seed(42)
import random
random.seed(42)

# Settings
import warnings
warnings.filterwarnings("ignore")
import time
pd.set_option('display.max_columns', None)

from joblib import dump, load

In [17]:
# deep learning algo
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras import backend
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# Fixing the seed for random number generators
tf.random.set_seed(42)

# dask
from dask_ml.model_selection import GridSearchCV as DaskGridSearchCV

# keras tuner
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

In [18]:
#!conda install -c conda-forge py-xgboost
import xgboost as xgb

#### Tree base function

In [19]:
# Database to store result
df_accuracy = pd.DataFrame(columns=['Model name', 'Model arch.', 'Train Accuracy', 'Test_Accuracy', 'Time'])

In [20]:
# Base models
def model_performance(name, model):
    start = time.time()
    
    # fit and make prediction 
    model.fit(X_train,y_train)
    y_hat_train = model.predict(X_train)
    y_hat_test = model.predict(X_test)
    
    # calculate accuracy
    accuracy_train = round(accuracy_score(y_train, y_hat_train),5)
    accuracy_test = round(accuracy_score(y_test, y_hat_test),5)
    training_time = round(time.time()-start,2)
    
    # print result
    #print('Train Accuracy:', accuracy_train)
    #print('Test  Accuracy:', accuracy_test)
    #print('Training time:', training_time,'s')
    
    # store result
    df_accuracy.loc[len(df_accuracy.index)] = [name, model, accuracy_train, accuracy_test, training_time]
    return df_accuracy.loc[len(df_accuracy.index)-1]

In [21]:
# Base models
def model_performance_fat(name, model):
    start = time.time()
    
    y_hat_train = model.predict(X_train)
    y_hat_test = model.predict(X_test)
    
    # calculate accuracy
    accuracy_train = round(accuracy_score(y_train, y_hat_train),5)
    accuracy_test = round(accuracy_score(y_test, y_hat_test),5)
    training_time = round(time.time()-start,2)
    
    # print result
    #print('Train Accuracy:', accuracy_train)
    #print('Test  Accuracy:', accuracy_test)
    #print('Training time:', training_time,'s')
    
    # store result
    df_accuracy.loc[len(df_accuracy.index)] = [name, model, accuracy_train, accuracy_test, training_time]
    return df_accuracy.loc[len(df_accuracy.index)-1]

In [55]:
def retrain_X(model):
    model.fit(X_all, Y, verbose = 2,
              early_stopping_rounds=10,
              eval_metric='aucpr',
              eval_set=[(X_test, y_test)],
             )

In [22]:
# export submission
def export_submission(model_name, model, note_version):
    print('Exporting ',model_name,'...')
    y_hat = np.round(model.predict(df_submit))
    df_to_submit = pd.DataFrame()
    df_to_submit['ID'] = ID_col
    df_to_submit['Overall_Experience'] = y_hat
    filename = 'submission_model_' + model_name + '_fromfile_' + note_version + '.csv'
    df_to_submit.to_csv(filename, index = False)
    print('Finish ',filename)

#### Data

In [23]:
df = pd.read_csv("df_imputed_from2.0.csv")
df.drop(columns=['Unnamed: 0',],inplace=True)
df.head()

,ID,Overall_Experience,Seat_Comfort,Seat_Class,Arrival_Time_Convenient,Catering,Platform_Location,Onboard_Wifi_Service,Onboard_Entertainment,Online_Support,Ease_of_Online_Booking,Onboard_Service,Legroom,Baggage_Handling,CheckIn_Service,Cleanliness,Online_Boarding,Gender,Customer_Type,Age,Type_Travel,Travel_Class,Travel_Distance,Departure_Delay_in_Mins,Arrival_Delay_in_Mins
0,98800001,0,Needs Improvement,Green Car,Excellent,Excellent,Very Convenient,Good,Needs Improvement,Acceptable,Needs Improvement,Needs Improvement,Acceptable,Needs Improvement,Good,Needs Improvement,Poor,Female,Loyal Customer,52.0,Business Travel,Business,272,0.0,5.0
1,98800002,0,Poor,Ordinary,Excellent,Poor,Needs Improvement,Good,Poor,Good,Good,Excellent,Needs Improvement,Poor,Needs Improvement,Good,Good,Male,Loyal Customer,48.0,Personal Travel,Eco,2200,9.0,0.0
2,98800003,1,Needs Improvement,Green Car,Needs Improvement,Needs Improvement,Needs Improvement,Needs Improvement,Good,Excellent,Excellent,Excellent,Excellent,Excellent,Good,Excellent,Excellent,Female,Loyal Customer,43.0,Business Travel,Business,1061,77.0,119.0
3,98800004,0,Acceptable,Ordinary,Needs Improvement,Acceptable,Needs Improvement,Acceptable,Needs Improvement,Acceptable,Acceptable,Acceptable,Acceptable,Acceptable,Good,Acceptable,Acceptable,Female,Loyal Customer,44.0,Business Travel,Business,780,13.0,18.0
4,98800005,1,Acceptable,Ordinary,Acceptable,Acceptable,Manageable,Needs Improvement,Good,Excellent,Good,Good,Good,Good,Good,Good,Good,Female,Loyal Customer,50.0,Business Travel,Business,1981,0.0,0.0


In [24]:
df_submit = pd.read_csv("df_submit_imputed_from2.0.csv")
df_submit.drop(columns=['Unnamed: 0',],inplace=True)
df_submit.head()

,ID,Seat_Comfort,Seat_Class,Arrival_Time_Convenient,Catering,Platform_Location,Onboard_Wifi_Service,Onboard_Entertainment,Online_Support,Ease_of_Online_Booking,Onboard_Service,Legroom,Baggage_Handling,CheckIn_Service,Cleanliness,Online_Boarding,Gender,Customer_Type,Age,Type_Travel,Travel_Class,Travel_Distance,Departure_Delay_in_Mins,Arrival_Delay_in_Mins
0,99900001,Acceptable,Green Car,Acceptable,Acceptable,Manageable,Needs Improvement,Excellent,Good,Excellent,Excellent,Excellent,Excellent,Good,Excellent,Poor,Female,Loyal Customer,36.0,Business Travel,Business,532,0.0,0.0
1,99900002,Extremely Poor,Ordinary,Good,Poor,Manageable,Acceptable,Poor,Acceptable,Acceptable,Excellent,Acceptable,Good,Acceptable,Excellent,Acceptable,Female,Disloyal Customer,21.0,Business Travel,Business,1425,9.0,28.0
2,99900003,Excellent,Ordinary,Excellent,Excellent,Very Convenient,Excellent,Excellent,Excellent,Needs Improvement,Needs Improvement,Needs Improvement,Needs Improvement,Good,Needs Improvement,Excellent,Male,Loyal Customer,60.0,Business Travel,Business,2832,0.0,0.0
3,99900004,Acceptable,Green Car,Excellent,Acceptable,Very Convenient,Poor,Acceptable,Excellent,Poor,Acceptable,Needs Improvement,Excellent,Excellent,Excellent,Poor,Female,Loyal Customer,29.0,Personal Travel,Eco,1352,0.0,0.0
4,99900005,Excellent,Ordinary,Extremely Poor,Excellent,Needs Improvement,Excellent,Excellent,Excellent,Excellent,Excellent,Acceptable,Excellent,Excellent,Excellent,Excellent,Male,Disloyal Customer,18.0,Business Travel,Business,1610,17.0,0.0


### 1/ Prep

In [25]:
unnecessary_cols = ['ID']
df.drop(columns = unnecessary_cols, inplace=True)

ID_col = df_submit['ID']
df_submit.drop(columns = unnecessary_cols, inplace=True)

In [26]:
#Convert rating to numeric first
standard_rating = ['Seat_Comfort',
                   'Arrival_Time_Convenient',
                   'Catering',
                   'Onboard_Wifi_Service',
                   'Onboard_Entertainment',
                   'Online_Support',
                   'Ease_of_Online_Booking',
                   'Onboard_Service',
                   'Legroom',
                   'Baggage_Handling',
                   'CheckIn_Service',
                   'Cleanliness',
                   'Online_Boarding']

for col in standard_rating:
    df[col].replace(['Excellent', 'Good', 'Acceptable', 'Needs Improvement', 'Poor', 'Extremely Poor'],
                    [5, 4, 3, 2, 1, 0], inplace = True)
    df_submit[col].replace(['Excellent', 'Good', 'Acceptable', 'Needs Improvement', 'Poor', 'Extremely Poor'],
                    [5, 4, 3, 2, 1, 0], inplace = True)
df['Platform_Location'].replace(['Very Convenient', 'Convenient', 'Manageable', 'Needs Improvement', 'Inconvenient','Very Inconvenient'], 
                                [5,4,3,2,1,0], inplace = True)
df_submit['Platform_Location'].replace(['Very Convenient', 'Convenient', 'Manageable', 'Needs Improvement', 'Inconvenient','Very Inconvenient'], 
                                [5,4,3,2,1,0], inplace = True)
df.head(5)

,Overall_Experience,Seat_Comfort,Seat_Class,Arrival_Time_Convenient,Catering,Platform_Location,Onboard_Wifi_Service,Onboard_Entertainment,Online_Support,Ease_of_Online_Booking,Onboard_Service,Legroom,Baggage_Handling,CheckIn_Service,Cleanliness,Online_Boarding,Gender,Customer_Type,Age,Type_Travel,Travel_Class,Travel_Distance,Departure_Delay_in_Mins,Arrival_Delay_in_Mins
0,0,2,Green Car,5,5,5,4,2,3,2,2,3,2,4,2,1,Female,Loyal Customer,52.0,Business Travel,Business,272,0.0,5.0
1,0,1,Ordinary,5,1,2,4,1,4,4,5,2,1,2,4,4,Male,Loyal Customer,48.0,Personal Travel,Eco,2200,9.0,0.0
2,1,2,Green Car,2,2,2,2,4,5,5,5,5,5,4,5,5,Female,Loyal Customer,43.0,Business Travel,Business,1061,77.0,119.0
3,0,3,Ordinary,2,3,2,3,2,3,3,3,3,3,4,3,3,Female,Loyal Customer,44.0,Business Travel,Business,780,13.0,18.0
4,1,3,Ordinary,3,3,3,2,4,5,4,4,4,4,4,4,4,Female,Loyal Customer,50.0,Business Travel,Business,1981,0.0,0.0


In [27]:
# Creating list of dummy columns
to_get_dummies1 = ['Seat_Class','Customer_Type','Type_Travel','Gender', 'Travel_Class',]

# Creating dummy variables
df = pd.get_dummies(data = df, columns = to_get_dummies1, drop_first = True)  
df_submit = pd.get_dummies(data = df_submit, columns = to_get_dummies1, drop_first = True)  

### Split

In [28]:
Y = df['Overall_Experience']

In [29]:
X = df.drop(columns = ['Overall_Experience'])

In [30]:
#X = X.values.astype(float)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=14)

scaler = preprocessing.StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

df_submit = scaler.transform(df_submit)

In [46]:
X_all = scaler.transform(X)

### XGBoost

### XGBoost params tuning

In [34]:
start = time.time()
random_grid = {'max_depth': np.arange(3,22,2),
               'learning_rate': [0.5,0.1, 0.05],
               'gamma': [0.0, 0.25, 1.0],
               'reg_lambda': [0.0, 1.0, 10.0],
               }
estimator = xgb.XGBClassifier(objective='binary:logistic', seed=14)
optimal_params = RandomizedSearchCV(estimator = estimator,
                                    param_distributions = random_grid,
                                    n_iter = 5, 
                                    cv = 3, 
                                    verbose=3, 
                                    random_state=42, 
                                    n_jobs = 10,
                                    scoring = 'roc_auc'
                                   )

optimal_params.fit(X_train, y_train)
print('Completed in:', round(time.time()-start,2))

Fitting 3 folds for each of 5 candidates, totalling 15 fits
[14:48:30] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Completed in: 43.01


In [35]:
print(optimal_params.best_params_)

{'reg_lambda': 0.0, 'max_depth': 9, 'learning_rate': 0.1, 'gamma': 1.0}


In [36]:
XGB_tune1 = optimal_params.best_estimator_

In [37]:
XGB_tune1.fit(X_train, y_train, verbose = 2,
          early_stopping_rounds=10,
          eval_metric='aucpr',
          eval_set=[(X_test, y_test)],
         )

[0]	validation_0-aucpr:0.98193
[2]	validation_0-aucpr:0.98556
[4]	validation_0-aucpr:0.98736
[6]	validation_0-aucpr:0.98822
[8]	validation_0-aucpr:0.98922
[10]	validation_0-aucpr:0.98972
[12]	validation_0-aucpr:0.99016
[14]	validation_0-aucpr:0.99050
[16]	validation_0-aucpr:0.99078
[18]	validation_0-aucpr:0.99103
[20]	validation_0-aucpr:0.99138
[22]	validation_0-aucpr:0.99165
[24]	validation_0-aucpr:0.99186
[26]	validation_0-aucpr:0.99210
[28]	validation_0-aucpr:0.99232
[30]	validation_0-aucpr:0.99248
[32]	validation_0-aucpr:0.99260
[34]	validation_0-aucpr:0.99284
[36]	validation_0-aucpr:0.99328
[38]	validation_0-aucpr:0.99344
[40]	validation_0-aucpr:0.99355
[42]	validation_0-aucpr:0.99366
[44]	validation_0-aucpr:0.99377
[46]	validation_0-aucpr:0.99383
[48]	validation_0-aucpr:0.99393
[50]	validation_0-aucpr:0.99401
[52]	validation_0-aucpr:0.99409
[54]	validation_0-aucpr:0.99415
[56]	validation_0-aucpr:0.99419
[58]	validation_0-aucpr:0.99425
[60]	validation_0-aucpr:0.99429
[62]	validati

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=1.0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.1, max_delta_step=0,
              max_depth=9, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=12,
              num_parallel_tree=1, predictor='auto', random_state=14,
              reg_alpha=0, reg_lambda=0.0, scale_pos_weight=1, seed=14,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [38]:
model_performance_fat('XGB_tune1', XGB_tune1)

Model name                                                XGB_tune1
Model arch.       XGBClassifier(base_score=0.5, booster='gbtree'...
Train Accuracy                                              0.97568
Test_Accuracy                                               0.95809
Time                                                           0.06
Name: 1, dtype: object

In [39]:
export_submission('XGB_tune1', XGB_tune1, '5.1')

Exporting  XGB_tune1 ...
Finish  submission_model_XGB_tune1_fromfile_5.1.csv


### XGBoost params tuning 2

In [59]:
start = time.time()
random_grid = {'max_depth': np.arange(5,30,2),
               'learning_rate': [0.12,0.11, 0.1,0.09],
               'gamma': [0.25, 1.0],
               'reg_lambda': [0.0, 1.0],
               }
estimator = xgb.XGBClassifier(objective='binary:logistic', seed=14)
optimal_params = RandomizedSearchCV(estimator = estimator,
                                    param_distributions = random_grid,
                                    n_iter = 15, 
                                    cv = 5, 
                                    verbose=3, 
                                    random_state=42, 
                                    n_jobs = 10,
                                    scoring = 'roc_auc'
                                   )

optimal_params.fit(X_train, y_train)
print('Completed in:', round(time.time()-start,2))

Fitting 5 folds for each of 15 candidates, totalling 75 fits
[15:14:53] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Completed in: 337.61


In [60]:
print(optimal_params.best_params_)

{'reg_lambda': 1.0, 'max_depth': 13, 'learning_rate': 0.12, 'gamma': 0.25}


In [61]:
XGB_tune2 = optimal_params.best_estimator_

In [62]:
XGB_tune2.fit(X_train, y_train, verbose = 2,
          early_stopping_rounds=10,
          eval_metric='aucpr',
          eval_set=[(X_test, y_test)],
         )

[0]	validation_0-aucpr:0.98788
[2]	validation_0-aucpr:0.98898
[4]	validation_0-aucpr:0.99000
[6]	validation_0-aucpr:0.99149
[8]	validation_0-aucpr:0.99178
[10]	validation_0-aucpr:0.99203
[12]	validation_0-aucpr:0.99259
[14]	validation_0-aucpr:0.99287
[16]	validation_0-aucpr:0.99313
[18]	validation_0-aucpr:0.99335
[20]	validation_0-aucpr:0.99345
[22]	validation_0-aucpr:0.99359
[24]	validation_0-aucpr:0.99373
[26]	validation_0-aucpr:0.99374
[28]	validation_0-aucpr:0.99340
[30]	validation_0-aucpr:0.99351
[32]	validation_0-aucpr:0.99366
[34]	validation_0-aucpr:0.99375
[35]	validation_0-aucpr:0.99377


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0.25, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.12, max_delta_step=0,
              max_depth=13, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=12,
              num_parallel_tree=1, predictor='auto', random_state=14,
              reg_alpha=0, reg_lambda=1.0, scale_pos_weight=1, seed=14,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [63]:
model_performance_fat('XGB_tune2', XGB_tune2)

Model name                                                XGB_tune2
Model arch.       XGBClassifier(base_score=0.5, booster='gbtree'...
Train Accuracy                                              0.97971
Test_Accuracy                                               0.95449
Time                                                           0.05
Name: 5, dtype: object

#### Retrain on all dataset

In [55]:
def retrain_X(model):
    model.fit(X_all, Y, verbose = 2,
              early_stopping_rounds=10,
              eval_metric='aucpr',
              eval_set=[(X_test, y_test)],
             )

In [64]:
retrain_X(XGB_tune2)

[0]	validation_0-aucpr:0.99167
[2]	validation_0-aucpr:0.99308
[4]	validation_0-aucpr:0.99367
[6]	validation_0-aucpr:0.99500
[8]	validation_0-aucpr:0.99574
[10]	validation_0-aucpr:0.99621
[12]	validation_0-aucpr:0.99663
[14]	validation_0-aucpr:0.99692
[16]	validation_0-aucpr:0.99713
[18]	validation_0-aucpr:0.99740
[20]	validation_0-aucpr:0.99760
[22]	validation_0-aucpr:0.99777
[24]	validation_0-aucpr:0.99787
[26]	validation_0-aucpr:0.99802
[28]	validation_0-aucpr:0.99823
[30]	validation_0-aucpr:0.99834
[32]	validation_0-aucpr:0.99845
[34]	validation_0-aucpr:0.99852
[36]	validation_0-aucpr:0.99858
[38]	validation_0-aucpr:0.99864
[40]	validation_0-aucpr:0.99872
[42]	validation_0-aucpr:0.99879
[44]	validation_0-aucpr:0.99882
[46]	validation_0-aucpr:0.99889
[48]	validation_0-aucpr:0.99899
[50]	validation_0-aucpr:0.99906
[52]	validation_0-aucpr:0.99915
[54]	validation_0-aucpr:0.99922
[56]	validation_0-aucpr:0.99929
[58]	validation_0-aucpr:0.99934
[60]	validation_0-aucpr:0.99939
[62]	validati

In [65]:
export_submission('XGB_tune2_retrain', XGB_tune2, '5.1')

Exporting  XGB_tune2_retrain ...
Finish  submission_model_XGB_tune2_retrain_fromfile_5.1.csv


### XGBoost params tuning 3

In [66]:
start = time.time()
random_grid = {'max_depth': np.arange(5,30,2),
               'learning_rate': [0.12,0.11, 0.1],
               'gamma': [0.0, 0.1, 0.25, 1.0],
               'reg_lambda': [0.0, 0.1, 1.0],
               }
estimator = xgb.XGBClassifier(objective='binary:logistic', seed=14)
optimal_params = RandomizedSearchCV(estimator = estimator,
                                    param_distributions = random_grid,
                                    n_iter = 10, 
                                    cv = 10, 
                                    verbose=3, 
                                    random_state=42, 
                                    n_jobs = 10,
                                    scoring = 'roc_auc'
                                   )

optimal_params.fit(X_train, y_train)
print('Completed in:', round(time.time()-start,2))

Fitting 10 folds for each of 10 candidates, totalling 100 fits
[15:30:46] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Completed in: 515.3


In [67]:
print(optimal_params.best_params_)

{'reg_lambda': 1.0, 'max_depth': 13, 'learning_rate': 0.1, 'gamma': 0.1}


In [68]:
XGB_tune3 = optimal_params.best_estimator_

In [69]:
XGB_tune3.fit(X_train, y_train, verbose = 2,
          early_stopping_rounds=10,
          eval_metric='aucpr',
          eval_set=[(X_test, y_test)],
         )

[0]	validation_0-aucpr:0.98786
[2]	validation_0-aucpr:0.98897
[4]	validation_0-aucpr:0.98996
[6]	validation_0-aucpr:0.99127
[8]	validation_0-aucpr:0.99162
[10]	validation_0-aucpr:0.99183
[12]	validation_0-aucpr:0.99196
[14]	validation_0-aucpr:0.99256
[16]	validation_0-aucpr:0.99274
[18]	validation_0-aucpr:0.99303
[20]	validation_0-aucpr:0.99315
[22]	validation_0-aucpr:0.99335
[24]	validation_0-aucpr:0.99349
[26]	validation_0-aucpr:0.99361
[28]	validation_0-aucpr:0.99375
[30]	validation_0-aucpr:0.99383
[32]	validation_0-aucpr:0.99400
[34]	validation_0-aucpr:0.99407
[36]	validation_0-aucpr:0.99416
[38]	validation_0-aucpr:0.99423
[40]	validation_0-aucpr:0.99429
[42]	validation_0-aucpr:0.99435
[44]	validation_0-aucpr:0.99441
[46]	validation_0-aucpr:0.99446
[48]	validation_0-aucpr:0.99451
[50]	validation_0-aucpr:0.99456
[52]	validation_0-aucpr:0.99461
[54]	validation_0-aucpr:0.99464
[56]	validation_0-aucpr:0.99467
[58]	validation_0-aucpr:0.99468
[60]	validation_0-aucpr:0.99472
[62]	validati

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0.1, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.1, max_delta_step=0,
              max_depth=13, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=12,
              num_parallel_tree=1, predictor='auto', random_state=14,
              reg_alpha=0, reg_lambda=1.0, scale_pos_weight=1, seed=14,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [70]:
model_performance_fat('XGB_tune3', XGB_tune3)

Model name                                                XGB_tune3
Model arch.       XGBClassifier(base_score=0.5, booster='gbtree'...
Train Accuracy                                              0.99187
Test_Accuracy                                               0.95825
Time                                                            0.1
Name: 6, dtype: object

#### Retrain on all dataset

In [71]:
retrain_X(XGB_tune3)

[0]	validation_0-aucpr:0.99169
[2]	validation_0-aucpr:0.99292
[4]	validation_0-aucpr:0.99347
[6]	validation_0-aucpr:0.99401
[8]	validation_0-aucpr:0.99503
[10]	validation_0-aucpr:0.99581
[12]	validation_0-aucpr:0.99623
[14]	validation_0-aucpr:0.99657
[16]	validation_0-aucpr:0.99683
[18]	validation_0-aucpr:0.99703
[20]	validation_0-aucpr:0.99722
[22]	validation_0-aucpr:0.99747
[24]	validation_0-aucpr:0.99762
[26]	validation_0-aucpr:0.99773
[28]	validation_0-aucpr:0.99783
[30]	validation_0-aucpr:0.99789
[32]	validation_0-aucpr:0.99807
[34]	validation_0-aucpr:0.99823
[36]	validation_0-aucpr:0.99833
[38]	validation_0-aucpr:0.99841
[40]	validation_0-aucpr:0.99849
[42]	validation_0-aucpr:0.99855
[44]	validation_0-aucpr:0.99860
[46]	validation_0-aucpr:0.99869
[48]	validation_0-aucpr:0.99874
[50]	validation_0-aucpr:0.99878
[52]	validation_0-aucpr:0.99884
[54]	validation_0-aucpr:0.99889
[56]	validation_0-aucpr:0.99898
[58]	validation_0-aucpr:0.99906
[60]	validation_0-aucpr:0.99912
[62]	validati

In [72]:
export_submission('XGB_tune3_retrain', XGB_tune3, '5.1')

Exporting  XGB_tune3_retrain ...
Finish  submission_model_XGB_tune3_retrain_fromfile_5.1.csv
